In [1]:
import os, sys, email
import numpy as np
import pandas as pd
import nltk
from nltk.metrics import *
import pprint, re, time, pickle

### 自然言語処理
import spacy 
nlp = spacy.load("en_core_web_sm")
from spacy.pipeline import Sentencizer
sentencizer = Sentencizer()
nlp.add_pipe(sentencizer)

pd.options.display.max_colwidth = 1000

In [2]:
# loadする

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_dict.pickle', mode='rb') as f:
    entity_dict = pickle.load(f)

with open('/Users/taroaso/myprojects/OpenIE/trec/output/entity_rows.pickle', mode='rb') as f:
    entity_rows = pickle.load(f)

from db import connect
engine = connect()
mail_df = pd.read_sql(sql='SELECT * FROM mail_3 ORDER BY docno ASC',con=engine, index_col=None)

In [12]:
# 新規エンティティ候補を登録する辞書をentity_candidate_dictとし，行をentity_candidate_rowsとする
entity_candidate_dict = {}
entity_candidate_rows = []

pattern = re.compile('\W+')

for mail in mail_df.values:
    docno = mail[0]
    body = mail[-1]
    doc = nlp(body)

    # 意味を持たないsentenceをフィルタリングする
    sentence_list = []
    for sentence in doc.sents:
        tokens = sentence.text.split()
        sentence = ' '.join(tokens)
        sentence = re.sub(r'[>|<]{1,}','',sentence)
        sentence = re.sub(r' {2,}',' ',sentence)
        if len(sentence) == 0 or sentence.isnumeric() or pattern.fullmatch(sentence):
            pass
        else:
            sentence_list.append(sentence)

    if len(sentence_list) == 0:
        pass
    else:
        tagme_start = time.time()
        sentences = ' '.join(sentence_list)
        doc = nlp(sentences)

    for ent in doc.ents:
        if ent.label_ not in ('DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY', 'CARDINAL', 'ORDINAL'):
            if ent.text in list(entity_dict.keys()):
                pass
            else:
                entity_candidate_dict[ent.text] = ent.label_
                entity_candidate_rows.append((docno,ent.text))

In [17]:
entity_dict

{&#39;ngeorgopoulos@city.ac.uk&#39;: {&#39;ngeorgopoulos@city.ac.uk&#39;: &#39;MAIL&#39;},
 &#39;Georgopoulos N&#39;: {&#39;Georgopoulos N&#39;: &#39;PERSON&#39;},
 &#39;14 Jul 2000&#39;: {&#39;14 Jul 2000&#39;: &#39;DATE&#39;},
 &#39;huftis@bigfoot.com&#39;: {&#39;huftis@bigfoot.com&#39;: &#39;MAIL&#39;},
 &#39;Karl Ove Hufthammer&#39;: {&#39;Karl Ove Hufthammer&#39;: &#39;PERSON&#39;},
 &#39;14 Jul 2001&#39;: {&#39;14 Jul 2001&#39;: &#39;DATE&#39;},
 &#39;klcc70@hotmail.com&#39;: {&#39;klcc70@hotmail.com&#39;: &#39;MAIL&#39;},
 &#39;lock mode&#39;: {&#39;lock mode&#39;: &#39;PERSON&#39;},
 &#39;29 Apr 2003&#39;: {&#39;29 Apr 2003&#39;: &#39;DATE&#39;},
 &#39;dtvc17@msn.com&#39;: {&#39;dtvc17@msn.com&#39;: &#39;MAIL&#39;},
 &#39;David Van Cott&#39;: {&#39;David Van Cott&#39;: &#39;PERSON&#39;},
 &#39;14 May 2003&#39;: {&#39;14 May 2003&#39;: &#39;DATE&#39;},
 &#39;azigan@fix.net&#39;: {&#39;azigan@fix.net&#39;: &#39;MAIL&#39;},
 &#39;Azigan&#39;: {&#39;Azigan&#39;: &#39;PERSON&#39;},
